# Cabled Mixer Calibration

The Cluster RF Modules (QCM-RF and QRM-RF) are equipped with integrated IQ mixers to perform up convert and downconvert RF signals. The output IQ mixers use a local oscillator (LO) with a frequency $\omega_{LO}$ to upconvert signals from the I and Q paths of frequency $\omega_{NCO}$ to output a signal at $\omega_{LO} + \omega_{NCO}$. However, since IQ mixers are inherently mathematically imperfect, the output also contains signal of frequency $\omega_{LO}$ (called leakage) and $\omega_{LO} - \omega_{NCO}$ (undesired sideband). With the firmware version `v0.7.0`, Qblox provides the capability to calibrate the output mixers of the RF modules using an input channel of a QRM-RF to suppress the LO leakage and undesired sideband. This eliminates the need for an external instrument (such as a spectrum analyzer).

Typical suppression of -35 dBc (compared to the desired sideband) up to 10 GHz can be achieved for inter module calibration.

Before proceeding, please make sure you have the following requirements satisfied:

#### Software Requirements
1. Cluster firmware version : v0.7.0 
2. `qblox-instruments` driver version : v0.12.0
3. `cma` Python package is installed in your Python environment via `pip install cma`
4. `mixer_calibration_utils.py` : This notebook assumes this file exists in the same directory as the notebook or is added to the Python import path accordingly. 

#### Hardware Requirements

1. Please make sure that all the individual modules slots are occupied (either with the module or a metallic flowblocker provided during delivery).
2. For each module and flowblocker, please tighten both the top screw and the bottom screw below the white clip to ensure effective grounding of the modules to the Cluster mainframe ground.  

# Setup

1. Please connect the output channel of a QCM-RF or a QRM-RF that you want to calibrate to a QRM-RF input channel via a RF cable. If you are calibrating a QRM-RF output channel, we recommend using the input channel of a different QRM-RF to achieve the typical suppression quoted above. It is possible to calibrate the output channel of a QRM-RF with its own input channel, however typical suppression is not guaranteed.
2. You could use a directional coupler to split the output to your device under test and to the input of a QRM-RF. 

# Imports

We import three functions from the `mixer_calibration_utils.py` :

1. `calibrate_lo` : Function used to calibrate the LO leakage for a particular output.
2. `calibrate_sideband` : Function used to calibrate the sideband leakage for a particular output and an associated sequencer with an NCO frequency.
3. `mixer_cal` : Wrapper function which calls both the above functions to achieve a full calibration for a particular set of output and sequencer.

In [1]:
from mixer_calibration_utils import (
    calibrate_lo,
    calibrate_sideband,
    mixer_cal,
)

# Connect to Instrument

In [2]:
from qblox_instruments import Cluster, ClusterType

ip_address = "192.0.2.72" # "192.168.0.2"

cluster = Cluster("cluster", identifier=ip_address)

cluster.reset()
# cluster.get_system_status()

ConnectionRefusedError: [Errno 111] Connection refused

In [3]:
# cluster.reboot()

# Calibration Procedure

Choose the module whose output needs to be calibrated and the QRM-RF whose input channel is used.

In [4]:
output_module = cluster.module1
input_module = cluster.module17

## Setting Output Attenuation

Before running the mixer calibration, ensure that there is 30 dB attenuation on the output. This can be achieved by setting `out{x}_att(30)` on the relevant output.

In [5]:
output_module.out0_att(30)

## LO Leakage Calibration

As mentioned above, we will use the function `calibrate_lo` to suppress the LO leakage. Please have a look at the docstring of this function in the `mixer_calibration_utils.py` file for descriptions of the keyword arguments.

The `lo_freq` is used to set the LO frequency of the output channel and the `lo_mismatch` keyword argument is used to set the LO frequency of the input (QRM-RF) module. QRM-RF input LO frequency would be 

$\omega_{LO_{QRM-RF}} = $ `lo_mismatch` $+ \omega_{LO}$   

The default LO mismatch of 77 MHz was chosen to avoid interference from common frequencies. However, you may change this if you have a 77 MHz signal that may interfere with the calibration.

If the `lo_freq` is not given or is specified as `None`, the function will query the LO frequency from the `module.out{x}_lo_freq` for QCM-RF or `module.out0_in0_lo_freq()` for QRM-RF.    

The mixer calibration parameters corresponding to the LO leakage suppresion namely `module.out{x}_offset_path0()`, `module.out{x}_offset_path1()`  are automatically set to the optimized values by the function.

**PLEASE NOTE** : `calibrate_lo` uses the SYNQ network (`wait_sync` command). Therefore please ensure that `sync_en` is set to `False` for all the sequencers. We use the `cluster.reset()` command to achieve this.

In [6]:
cluster.reset()

In [7]:
qubits = ['q16','q17','q18','q19','q20','q21','q22','q23','q24','q25']
lo_list = [4.25025000e+09, 5.41650000e+09, 4.83750000e+09, 5.14600000e+09,
       4.40025000e+09, 5.21475000e+09, 4.72275000e+09, 5.27716667e+09,
       4.63350000e+09, 5.31275000e+09]
qubit_lo = dict(zip(qubits, lo_list))
qubit_lo

{'q16': 4250250000.0,
 'q17': 5416500000.0,
 'q18': 4837500000.0,
 'q19': 5146000000.0,
 'q20': 4400250000.0,
 'q21': 5214750000.0,
 'q22': 4722750000.0,
 'q23': 5277166670.0,
 'q24': 4633500000.0,
 'q25': 5312750000.0}

In [8]:
offset_I, offset_Q = calibrate_lo(
    cal_module=output_module,   # Module to calibrate
    cal_output=0,                  # Output to calibrate
    input_module=input_module,  # Module to use for measurement
    lo_freq=None,                # (Default `None`) LO frequency to calibrate. Omit or use `None` to get from hardware.
    input_sequencer=0,             # (Default 0) One of two sequencers in the input module that should be used to calibrate.
    input_sequencer2=1,            # (Default 1) The second of two sequencers in the input module that should be used to calibrate.
    lo_mismatch=77e6,              # (Default 77e6) The difference between the LO of the `input module` and mixer that is being calibrated.
    verbose=True,                  # (Default False) Displays the measurement results during calibration and optimal values after calibration.
)

print(f"Best DC offsets : {offset_I}, {offset_Q}" + " "*100)

RuntimeError: ('"Undefined header;SLOT17:LO:SH:ENA 1 "', 'setting cluster_module1_out0_lo_en to True')

## Undesired Sideband Suppression

As mentioned in the Imports section, we will use the function `calibrate_sideband` to suppress the undesired sideband. Please have a look at the docstring of this function in the `mixer_calibration_utils.py` file for descriptions of the keyword arguments.

Please make sure that the value of `cal_sequencer` is different from the `input_sequencer` and `input_sequencer2` since the former is the sequencer that will be calibrated and the latter two sequencers will be used to calibrate the `cal_sequencer`.

The `cal_freq` is the total frequency $\omega_{LO} + \omega_{NCO}$ of the output signal. If specified, the function will query the LO frequency from the `module.out{x}_lo_freq` for QCM-RF or `module.out0_in0_lo_freq()` for QRM-RF and set the NCO frequency as the difference of the `cal_freq` and LO frequency.

If `cal_freq` is not specified or is `None`, $\omega_{NCO}$ is retrieved from the `cal_sequencer`.

Please follow the same rule when setting the `lo_mismatch` value as in `calibrate_lo`.  

The mixer calibration parameters corresponding to the undesired sideband suppresion namely `module.sequencer{x}.mixer_corr_gain_ratio()`, `module.sequencer{x}.mixer_corr_phase_offset_degree()`  are automatically set to the optimized values by the function.

In [ ]:
amp_ratio, phase_offset = calibrate_sideband(
    cal_module=output_module,   # Module to calibrate
    cal_output=0,                  # Output to calibrate
    input_module=input_module,  # Module to use for measurement
    cal_freq=4.422661e9,           # (Default `None`) Desired sideband frequency to calibrate. Omit or use `None` to get from hardware.
    cal_sequencer=2,               # (Default 2) One of two sequencers in the input module that should be used to calibrate.
    input_sequencer=0,             # (Default 0) One of two sequencers in the input module that should be used to calibrate.
    input_sequencer2=1,            # (Default 1) The second of two sequencers in the input module that should be used to calibrate.
    lo_mismatch=77e6,              # (Default 77e6) The difference between the LO of the `input module` and mixer that is being calibrated.
    verbose=True,                  # (Default False) Displays the measurement results during calibration and optimal values after calibration.
)

print(f"Best amplitude ratio and phase offset : {amp_ratio}, {phase_offset}" + " "*100)

## Wrapper Function for Full Mixer Calibration

Using the convenience function `mixer_calibration_utils.mixer_cal` one can calibrate the LO leakage and suppress the undesired sideband together. The keyword arguments for this are the same as for the respective arguments of  `calibrate_lo` and `calibrate_sideband`.

In [9]:
#Reset cluster to disable sync_en()
cluster.reset() 

In [10]:
offset_I, offset_Q, amp_ratio, phase_offset = mixer_cal(
    cal_module=cluster.module1,
    cal_output=0,
    input_module=cluster.module17,
)

print(f"Best DC offsets : {offset_I}, {offset_Q}" + " "*100)
print(f"Best amplitude ratio and phase offset : {amp_ratio}, {phase_offset}" + " "*100)

RuntimeError: ('"Undefined header;SLOT17:LO:SH:ENA 1 "', 'setting cluster_module1_out0_lo_en to True')

In [ ]:
# stop all sequencers and close the cluster connection 
cluster.stop_sequencer()
cluster.close()